<img src="https://i.ibb.co/hcrKx44/Weekly-Challenge-Banner.png" >

# Weekly Challenge 1
## Description

Welcome to the first challenge! This week, you will perform a _data wrangling_ task with [pandas](https://pandas.pydata.org/), a standard data science library.

## About the data

The data for this challenge includes demographic statistics for Swiss districts and communes in 2019. 

More details about the dataset can be found [here](https://www.bfs.admin.ch/bfs/fr/home/statistiques/population/effectif-evolution/repartition-territoriale.assetdetail.13707350.html).

You can see that some of the rows do not actually contain data from a commune but statistics for the cantons themselves (e.g., "- Zürich" in the first line).

## The task

Some cities share the same name (e.g., **Ecublens** in Vaud and in Fribourg). Your task is to find the population of the canton with the most reduplicated commune names!

In [1]:
import pandas as pd

# Load the data
df = pd.read_excel('data/demographic_stats.xlsx', skiprows=4, skipfooter=6, sheet_name='2019', 
                   usecols=list(range(4)), names=['Commune', 'Population', 'Births', 'Deaths'], 
                   index_col='Commune')

# Load canton name abbreviations
abbreviations = pd.read_csv('data/abbreviations.txt', delimiter = '\t', index_col=0)

ipykernel_launcher:6: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.


In [2]:
df.iloc[2075:2087]

,Population,Births,Deaths
Commune,,,
......5583 Crissier,7930,88,29
......5589 Prilly,12399,134,109
......5591 Renens (VD),20927,226,130
......5624 Bussigny,8736,109,37
......5627 Chavannes-près-Renens,7727,113,32
......5635 Ecublens (VD),12863,136,73
......5648 Saint-Sulpice (VD),4668,41,28
......5651 Villars-Sainte-Croix,962,8,5
>> District de la Riviera-Pays-d'Enhaut,84980,790,723


In [3]:
abbreviations.head()

,Canton
Abbreviation,
ZH,Zürich
BE,Bern
LU,Lucerne
UR,Uri
SZ,Schwyz


## Solution

There are actually two different solutions. The first is very straightfoward: start with all communes, then only keep the commune whose name has a canton indication in parentheses (e.g., `Ecublens (VD)`), then only keep the canton indication part, count the number of occurrences of each canton abbreviation, find the canton that appears most often, and finally use the initial data to extract the population of that canton. The code below does this step-by-step:

### First way: assume that a canton indication in parentheses means that the commune name is not unique

#### The short way

In [4]:
df.loc['- ' + abbreviations.loc[df[df.index.str.contains('\(')].index.map(lambda x: x[-3:-1]).value_counts().index[0]].Canton].Population

694207

#### The long way:

In [5]:
# Step 1: only keep rows that contain an opening parenthesis
communes = df[df.index.str.contains('\(')]
communes.tail(11)

,Population,Births,Deaths
Commune,,,
......6087 Saint-Martin (VS),825,1,8
......6202 Wiler (Lötschen),565,2,4
......6293 Stalden (VS),1086,8,16
......6452 Cressier (NE),1887,19,12
......6454 Hauterive (NE),2623,32,17
......6608 Carouge (GE),22458,267,111
......6619 Corsier (GE),2108,18,11
......6704 Châtillon (JU),481,3,3
......6741 Le Bémont (JU),317,3,2


As we can see, some communes have parentheses in their name even though the value in-between parentheses doesn't stand for a canton. Thankfully, this is not a problem since we will only consider the value in-between parentheses that occurs most often.

In [6]:
# Step 2: extract value in-between parentheses from commune name
cantons = communes.index.map(lambda x: x[-3:-1])
cantons

Index(['ZH', 'ZH', 'ZH', 'ZH', 'ZH', 'ZH', 'ZH', 'ZH', 'ZH', 'ZH',
       ...
       'en', 'VS', 'NE', 'NE', 'GE', 'GE', 'JU', 'JU', 'JU', 'JU'],
      dtype='object', name='Commune', length=173)

In [7]:
# Step 3: rank by number of occurences
ranking = cantons.value_counts()
ranking.head()

AG    21
BE    20
ZH    16
FR    16
VD    15
Name: Commune, dtype: int64

In [8]:
# Step 4: keep top entry of ranking
abbreviation = ranking.index[0]

# Step 5: map abbreviation to canton name
canton = abbreviations.loc[abbreviation].Canton
canton

'Aargau'

And so the canton with the most reduplicated commune names is **Aargau**.

In [9]:
# Step 6: extract population of this canton
# The dataframe contains entries like '- Vaud' that 
# contain the total stats for each canton
population = df.loc['- ' + canton].Population
population

694207

And there we go, `694207` is the first possible solution.

### Second way: check if the commune names with parentheses are actually not unique

One could also manually check if a commune name is unique or not, and perform the same ranking as above but without the assumption that the canton indication between parentheses means that the commune name is unique. As it turns out, this is the better way to do it: the commune `Renens` for instance, denoted as `Renens (VD)`, only exists once in Switzerland. 

In [10]:
# Step 1: map commune name to "true" name by removing the leading dots and numbers
# and the trailing indication in-between parentheses if it exists
def map_name(x):
    if x.startswith('-'):
        # do not change canton names
        return x

    # otherwise, remove dots and zip codes
    cleaned = ' '.join(x.split(' ')[1:])

    if '(' in cleaned:
        # remove everything after the opening parenthesis
        return cleaned[:cleaned.index('(')-1]
    else:
        return cleaned

In [11]:
# Step 2: map commune names using the above method and rank them
counts = df.index.map(map_name).value_counts()
counts.head()

Rickenbach    5
Aesch         3
Oberdorf      3
Buchs         3
Wald          3
Name: Commune, dtype: int64

In [12]:
# Step 3: only keep commune names that occur more than once
duplicated = counts[counts > 1]
duplicated = df[df.index.map(lambda x: map_name(x) in duplicated.index)]

# Step 4: do the same as in the first approach, but only with the duplicated communes
# The assumption here is that even though a canton indication in a commune name might
# not be sufficient for it to be non-unique, it is definitely necessary
counts = duplicated.index.map(lambda x: x[-3:-1]).value_counts()
counts.head()

FR    11
BE    10
ZH    10
AG     9
SG     8
Name: Commune, dtype: int64

As we can see, the top canton is now **Fribourg**.

In [13]:
df.loc['- ' + abbreviations.loc[counts.index[0]].Canton].Population

318714

And `318714` is the second possible solution.

## Congratulations to everyone that got the answer!